# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#Weather URL
weather_url = "http://api.openweathermap.org/data/2.5/weather?"

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
class Keys:
    is_fetch_ok = "Is_Fetch_Ok"
    city_id     = "City_ID"
    city        = "City"
    cloudiness  = "Cloudiness"
    country     = "Country"
    date        = "Date"
    humidity    = "Humidity"
    lat         = "Lat"
    lng         = "Lng"
    max_temp    = "Max Temp"
    wind_speed  = "Wind Speed"   

## Generate Cities List

In [3]:

# List for holding lat_lngs and cities
lat_lngs     = []

#Keeping track of processed cities
cities       = []

#Stores all weather-relevant data for a given set of coordinates
weather_container = []

#************************************#
#*            Test Vars             *#
#************************************#
geographical_span = 15 #was 1500 TODO change back
is_test = True     #TODO remove
test_humidity = 70 #TODO remove
#************************************#
#*            Test Vars             *#
#************************************#

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=geographical_span)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=geographical_span)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    
    city_name    = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country_code = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code

    # If the city is unique, then add it to 'cities'
    if city_name not in cities:
        
        weather_hash  = {
                             Keys.is_fetch_ok : False,
                             Keys.city_id     : None, 
                             Keys.city        : city_name, 
                             Keys.cloudiness  : None, 
                             Keys.country     : country_code.upper(),
                             Keys.date        : None,
                             Keys.humidity    : None,
                             Keys.lat         : lat_lng[0],
                             Keys.lng         : lat_lng[1],
                             Keys.max_temp    : None,
                             Keys.wind_speed  : None
                        }        
        
        weather_container.append(weather_hash)
        cities.append(city_name)
        
# Print the city count to confirm (sufficient) count
print(f"The pseudo-random number of unique cities in this unit of execution is {len(cities)}")

The pseudo-random number of unique cities in this unit of execution is 15


In [4]:
def print_city_log(status_code : int, 
                   city_name   : str       = None, 
                   rec_set     : int       = None, 
                   success_fetch_cntr :int = None) -> None:
    """
    Prints running log of city number and name, if and only if, if the city contains 500+ people, otherwise
    the city name will not be printed

    :param int status_code: The status code of the Response
    :param str city_name: The city name for a given set of coordinates
    :param int rec_set: The count of the processing record set
    :param int success_fetch_cntr: The current record fetch count
    """        
    if(status_code == 200):
        print(f"Processing Record {success_fetch_cntr} of Set {rec_set} | {city_name}")
    else:
        print("City not found. Skipping...")

In [5]:
def print_line_sep() -> None:
    
    """
    Prints a line delimiter
    """       
    
    print("----------------------------")

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
# Build query URL
  
#out_cols = ["City_ID","Cloudiness","Country","Date","Humidity","Lat","Lng","Max Temp","Wind Speed"]

total_success_fetch_cntr = 0
rec_set_cntr = 0
set_cntr = 1

print("Beginning Data Retrieval")
print_line_sep()

for i in range(len(weather_container)):
    
    weather_hash = weather_container[i]
    city_name    = weather_hash[Keys.city]
    weather_query_url = weather_url + "appid=" + weather_api_key + "&q=" + city_name
    
    if (i + 1) % 60 != 0:
        
        weather_response  = requests.get(weather_query_url)
        status_code       = weather_response.status_code

        if status_code == 200 :

            weather_json = weather_response.json()
            
            rec_set_cntr += 1
            
            weather_hash[Keys.is_fetch_ok] = True
            weather_hash[Keys.city_id]     = total_success_fetch_cntr #id starts at 0 not 1
            total_success_fetch_cntr += 1
            
            try:
                
                #weather_hash[date]        = "" if weather_json       == None  or \
                                                  #weather_json["dt"] == ""    or \
                                                  #weather_json["dt"] == None else str(weather_json["dt"])
                
                weather_hash[Keys.date]        = weather_json["dt"]
                
                weather_hash[Keys.max_temp]    = weather_json["main"]["temp_max"]
                weather_hash[Keys.humidity]    = weather_json["main"]["humidity"]      
                weather_hash[Keys.wind_speed]  = weather_json["wind"]["speed"]
                weather_hash[Keys.cloudiness]  = weather_json["clouds"]["all"]
                
                if is_test:
                    time.sleep(1)              
            
            except IndexError as ie:
                print(f"Index Exception occured {ie}")
                break
                
            except Exception as e:
                print(f"Index Exception occured {e}")
                break                
                
            else:
                      
                print_city_log(status_code,
                               city_name,
                               set_cntr,
                               rec_set_cntr)
                
        else:
            print_city_log(status_code)
    
    else:
        set_cntr    += 1
        rec_set_cntr = 0
        time.sleep(60)

print_line_sep()
print("Data Retrieval Complete")
print_line_sep()


Beginning Data Retrieval
----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | saint anthony
Processing Record 3 of Set 1 | georgetown
Processing Record 4 of Set 1 | deputatskiy
Processing Record 5 of Set 1 | saint-philippe
Processing Record 6 of Set 1 | ndele
Processing Record 7 of Set 1 | chokurdakh
Processing Record 8 of Set 1 | havoysund
Processing Record 9 of Set 1 | pevek
Processing Record 10 of Set 1 | cayenne
Processing Record 11 of Set 1 | pizhanka
Processing Record 12 of Set 1 | mayumba
Processing Record 13 of Set 1 | lagoa
Processing Record 14 of Set 1 | east london
Processing Record 15 of Set 1 | ancud
----------------------------
Data Retrieval Complete
----------------------------


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [24]:
# Put data into DataFrame
raw_results = pd.DataFrame(weather_container)
fetched_results_df = raw_results.loc[raw_results[Keys.is_fetch_ok] == True][[Keys.city_id,
                                                                             Keys.city,
                                                                             Keys.cloudiness,
                                                                             Keys.country,
                                                                             Keys.date,
                                                                             Keys.humidity,
                                                                             Keys.lat,
                                                                             Keys.lng,
                                                                             Keys.max_temp,
                                                                             Keys.wind_speed]].set_index(Keys.city_id)

#Export the DataFrame to CSV
fetched_results_df.to_csv(output_data_file,
                          header=True,
                          mode='w',
                          index=True)

temp_dataframe = fetched_results_df.copy(deep = True) #Not in situ

temp_dataframe = temp_dataframe.reset_index()

temp_dataframe = temp_dataframe.astype({Keys.city_id:    int,
                                        Keys.humidity:   int,
                                        Keys.cloudiness: int,
                                        Keys.date: 'str'})

temp_dataframe = temp_dataframe.astype({Keys.lat:float}).round(2)
temp_dataframe = temp_dataframe.astype({Keys.lng:float}).round(2)

temp_dataframe[Keys.date].replace('.0', '', inplace = True) 

temp_dataframe[[Keys.city,
                Keys.lat,
                Keys.lng,
                Keys.max_temp,
                Keys.humidity,
                Keys.cloudiness,
                Keys.wind_speed,
                Keys.country,
                Keys.date]].head(10) 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-66.02,-59.07,277.96,65,20,7.20,AR,1635302568
1,saint anthony,52.51,-53.74,284.43,55,40,1.79,CA,1635302637
2,georgetown,-5.13,-9.88,302.12,80,20,0.51,SH,1635302598
3,deputatskiy,72.61,140.83,258.58,92,100,1.17,RU,1635302672
4,saint-philippe,-61.81,68.70,294.94,74,15,4.58,RE,1635302330
5,ndele,8.19,20.89,294.11,89,96,0.82,CF,1635302941
6,chokurdakh,89.33,147.22,251.36,97,99,1.83,RU,1635302645
7,havoysund,71.77,25.08,273.64,60,91,11.78,NO,1635302706
8,pevek,87.80,170.71,261.57,93,100,5.90,RU,1635302566
9,cayenne,10.08,-44.74,300.42,87,20,2.24,GF,1635302946


In [25]:
temp_dataframe = None #Clean up

In [26]:
#Display discriptive fundamental statistics visualization of successfully fetched results

fetched_results_df[[Keys.city,
                    Keys.lat,
                    Keys.lng,
                    Keys.max_temp,
                    Keys.humidity,
                    Keys.cloudiness,
                    Keys.wind_speed,
                    Keys.country,
                    Keys.date]].describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,1.500000e+01
mean,16.564922,25.552967,282.646667,82.866667,72.866667,4.430667,1.635303e+09
std,56.909241,80.922667,16.073295,13.190184,36.978501,3.619160,1.855936e+02
min,-68.284143,-102.542532,251.360000,55.000000,15.000000,0.510000,1.635302e+09
25%,-24.599722,-37.045359,273.235000,77.000000,30.000000,1.710000,1.635303e+09
50%,10.076579,20.892588,284.500000,89.000000,96.000000,3.280000,1.635303e+09
75%,64.457988,60.560669,294.525000,91.500000,100.000000,6.550000,1.635303e+09
max,89.328109,170.714420,302.120000,97.000000,100.000000,11.780000,1.635303e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [27]:
#  Get the indices of cities that have humidity over 100%.

humidity_limit = test_humidity  ##Test Humidity [Test Variable] set above
print(f"\nIdentifying records with humidity over {humidity_limit}% to remove")

excessive_humidity_df = fetched_results_df.loc[fetched_results_df[Keys.humidity] > humidity_limit].reset_index()[[Keys.city_id,Keys.country,Keys.city,Keys.humidity]]  \
                                                                                                  .astype({Keys.city_id: 'int'})                                                                       
excessive_humidity_df.head(15)


Identifying records with humidity over 70% to remove


,City_ID,Country,City,Humidity
0,2,SH,georgetown,80
1,3,RU,deputatskiy,92
2,4,RE,saint-philippe,74
3,5,CF,ndele,89
4,6,RU,chokurdakh,97
5,8,RU,pevek,93
6,9,GF,cayenne,87
7,10,RU,pizhanka,93
8,11,GA,mayumba,91
9,12,PT,lagoa,89


In [21]:
#Re-configure City ID type from float to int to match visualization requirement
excessive_humidity_df = excessive_humidity_df.astype({Keys.city_id: 'int'}) #.set_index(city_id)


In [30]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
idxs_to_drop = list((x for x in list(excessive_humidity_df[Keys.city_id])))

# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = fetched_results_df.drop(fetched_results_df.index[idxs_to_drop], inplace = False )

clean_city_data = clean_city_data.reset_index()

clean_city_data = clean_city_data.astype({Keys.city_id:    int,
                                          Keys.humidity:   int,
                                          Keys.cloudiness: int,
                                          Keys.date: 'str'})

clean_city_data = clean_city_data.astype({Keys.lat:float}).round(2)
clean_city_data = clean_city_data.astype({Keys.lng:float}).round(2)

clean_city_data[Keys.date].replace('.0', '', inplace = True) 

clean_city_data[[Keys.city,
                 Keys.lat,
                 Keys.lng,
                 Keys.max_temp,
                 Keys.humidity,
                 Keys.cloudiness,
                 Keys.wind_speed,
                 Keys.country,
                 Keys.date]].head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-66.02,-59.07,277.96,65,20,7.20,AR,1635302568
1,saint anthony,52.51,-53.74,284.43,55,40,1.79,CA,1635302637
2,havoysund,71.77,25.08,273.64,60,91,11.78,NO,1635302706


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression